**1. Data Analysyis** - Handling Values of Data

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd

train_df = pd.read_csv("/content/drive/MyDrive/amazon/dataset/test.csv")#train.csv path
test_df = pd.read_csv("/content/drive/MyDrive/amazon/dataset/train.csv")#test.csv path
print(train_df.head())

In [ ]:
print(test_df.head())

In [ ]:
print(train_df.isnull().sum())

In [ ]:
print(test_df.isnull().sum())

In [ ]:
print(train_df["entity_name"].value_counts())

Installing Requirements

In [ ]:
!pip install boto3


In [ ]:
!pip install tensorflow


In [ ]:
!pip install lightgbm


In [ ]:
!pip install scikit-learn


In [ ]:
!pip install requests


In [ ]:
!pip install pandas


In [ ]:
!pip install tqdm


In [ ]:
!pip install s3fs

In [ ]:
import boto3
import tensorflow as tf
import lightgbm as lgb
import requests
import pandas as pd
import sklearn
import tqdm


**2. Data Preprocessing** - Loading images with AWS S3 buckets coz large data

In [ ]:
import boto3

# Initialize S3 client with specific credentials
s3 = boto3.client(
    's3',
    aws_access_key_id='Your-access',
    aws_secret_access_key='Your-secret access',
    region_name='us-east-1'
)


Downloading images with image url from each row at train.csv data

In [ ]:
import boto3
import pandas as pd
import requests
import os
from tqdm import tqdm

# Initialize S3 client with specific credentials
# Ensure the credentials have read access to the bucket and the file.
s3 = boto3.client(
    's3',
    aws_access_key_id='Your-access',
    aws_secret_access_key='Your-secret access',
    region_name='us-east-1'
)
bucket_name = 'Your bucket name'

# Create the directory if it doesn't exist
if not os.path.exists('train_images'):
    os.makedirs('train_images')

# Load train.csv
train_df = pd.read_csv(f's3://{bucket_name}/train.csv', storage_options={"key": "Your-access", "secret": "Your-secret access"})

# Download images and upload to S3
for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
    img_url = row['image_link']
    img_name = f"train_images/{row['group_id']}_{idx}.jpg"

    try:
        # Download the image
        img_data = requests.get(img_url).content

        # Save the image to local directory
        with open(img_name, 'wb') as handler:
            handler.write(img_data)

        # Upload the image to S3
        s3.upload_file(img_name, bucket_name, img_name)
    except Exception as e:
        print(f"Error downloading {img_url}: {e}")


**3. Feature Extraction based on ResNet50**  - which can be fed into a classifier (like LightGBM)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

# Load ResNet50 without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add global pooling layer to flatten the features
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
model = Model(inputs=base_model.input, outputs=x)


In [ ]:
import tensorflow as tf
import boto3
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from io import BytesIO

# S3 client to access images
s3 = boto3.client('s3',
                  aws_access_key_id='Your-access',
                  aws_secret_access_key='Your-secret access')

bucket_name = 'Your bucket name'
prefix = 'train_images/'  # Folder where images are stored

# Function to list all files in the specified S3 bucket folder
def list_files_in_s3_folder(bucket_name, prefix):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    if 'Contents' in response:
        files = [obj['Key'] for obj in response['Contents']]
        return files
    else:
        print(f"No files found in {bucket_name}/{prefix}")
        return []

# Function to load image from S3
def load_image_from_s3(image_path, target_size=(224, 224)):
    try:
        img_obj = s3.get_object(Bucket=bucket_name, Key=image_path)
        img = load_img(BytesIO(img_obj['Body'].read()), target_size=target_size)
        img_array = img_to_array(img)
        img_array = img_array / 255.0  # normalize
        return img_array
    except s3.exceptions.NoSuchKey as e:
        print(f"Error: {image_path} not found in bucket {bucket_name}")
        return None

# List all files in the 'train_images' folder
files_in_s3 = list_files_in_s3_folder(bucket_name, prefix)

# Test loading an image if the file exists
test_image_path = 'train_images/107694_10032.jpg'  # Replace with actual image path

if test_image_path in files_in_s3:
    sample_image = load_image_from_s3(test_image_path)
    if sample_image is not None:
        print(f"Loaded image: {test_image_path}, shape: {sample_image.shape}")
else:
    print(f"Image {test_image_path} does not exist in S3 bucket {bucket_name}")


In [ ]:
import tensorflow as tf
import boto3
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from io import BytesIO

# S3 client to access images
s3 = boto3.client('s3',
                  aws_access_key_id='Your-access',
                  aws_secret_access_key='Your-secret access')

bucket_name = 'bucket name'
prefix = 'train_images/'  # Folder where images are stored

# Load train.csv
train_df = pd.read_csv('s3://bucket name/train.csv')

# Load image from S3
def load_image_from_s3(image_path, target_size=(224, 224)):
    img_obj = s3.get_object(Bucket=bucket_name, Key=image_path)
    img = load_img(BytesIO(img_obj['Body'].read()), target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # normalize
    return img_array

# Prepare image paths for the training data
def prepare_images(image_paths):
    images = []
    for path in image_paths:
        img = load_image_from_s3(path)
        if img is not None:
            images.append(img)
    return np.array(images)

# ResNet50 Feature Extractor
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
model = Model(inputs=base_model.input, outputs=x)

# Prepare image paths for training data
image_paths = [prefix + str(group_id) + '_' + str(idx) + '.jpg' for idx, group_id in enumerate(train_df['group_id'])]

# Preprocess images
train_images = prepare_images(image_paths)

# Extract features from the images
image_features = model.predict(train_images)

# Flatten and convert features to array
image_features = np.reshape(image_features, (image_features.shape[0], -1))

# Now map the features to entity_value (target labels)
train_labels = train_df['entity_value'].values



In [ ]:
print(image_paths[:5])  # Print first 5 image paths to check


In [ ]:
# Test saving an image from S3
save_image_from_s3('train_images/748919_0.jpg', 'test_image.jpg')

# Open the saved image file and verify it's correctly loaded
from PIL import Image
img = Image.open('test_image.jpg')
img.show()  # This should display the image if it's valid


In [ ]:
def load_image_from_s3(image_path, target_size=(224, 224)):
    try:
        img_obj = s3.get_object(Bucket=bucket_name, Key=image_path)
        img_data = img_obj['Body'].read()
        print(f"Image data length: {len(img_data)} bytes")  # Check the size of the image data
        img = load_img(BytesIO(img_data), target_size=target_size)
        img_array = img_to_array(img)
        img_array = img_array / 255.0  # Normalize
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None


**4. Feature Storage** - Batch processing & feature extraction

In [ ]:
import time

def process_images_in_batches(image_paths, batch_size=32):
    features = []
    start_time = time.time()  # Start time
    for batch_start in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[batch_start:batch_start + batch_size]
        print(f"Processing batch {batch_start // batch_size + 1} of size {len(batch_paths)}")  # Debugging
        batch_images = prepare_images(batch_paths)
        if len(batch_images) > 0:
            batch_features = model.predict(batch_images)
            features.append(batch_features)
    end_time = time.time()  # End time
    print(f"Total processing time: {end_time - start_time} seconds")
    return np.vstack(features)

# Process images in batches
train_images = process_images_in_batches(image_paths)


In [ ]:
def process_images_in_batches(image_paths, batch_size=32):
    features = []
    start_time = time.time()  # Start time
    for batch_start in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[batch_start:batch_start + batch_size]
        print(f"Processing batch {batch_start // batch_size + 1} of size {len(batch_paths)}")  # Debugging
        batch_images = prepare_images(batch_paths)
        print(f"Batch images shape: {batch_images.shape if batch_images is not None else 'None'}")
        if len(batch_images) > 0:
            batch_features = model.predict(batch_images)
            print(f"Batch features shape: {batch_features.shape}")
            features.append(batch_features)
    end_time = time.time()  # End time
    print(f"Total processing time: {end_time - start_time} seconds")
    return np.vstack(features) if len(features) > 0 else None


In [ ]:
train_labels = train_df['entity_value'].values  # Assuming they are numerical


In [ ]:
def prepare_images(image_paths):
    images = []
    for path in image_paths:
        img = load_image_from_s3(path)
        if img is not None:
            print(f"Image shape: {img.shape}")  # Check image dimensions
            images.append(img)
    return np.array(images)


In [ ]:
prefix = 'train_images/'  # Your folder path in S3
image_paths = [prefix + str(group_id) + '_' + str(idx) + '.jpg' for idx, group_id in enumerate(train_df['group_id'])]


In [ ]:
train_df = pd.read_csv('s3://bucket name/train.csv')


In [ ]:
print(image_paths[:5])  # Check first 5 image paths


In [ ]:
def test_load_image_from_s3(image_path, target_size=(224, 224)):
    try:
        img_obj = s3.get_object(Bucket=bucket_name, Key=image_path)
        img = load_img(BytesIO(img_obj['Body'].read()), target_size=target_size)
        img.show()  # Display image to ensure it’s valid
        print(f"Successfully loaded image: {image_path}")
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")

# Test with a specific image path
test_load_image_from_s3('train_images/748919_0.jpg')


In [ ]:
def prepare_images(image_paths):
    images = []
    for path in image_paths:
        try:
            img = load_image_from_s3(path)
            if img is not None:
                images.append(img)
        except Exception as e:
            print(f"Error processing image {path}: {e}")
    return np.array(images) if len(images) > 0 else None


In [ ]:
def process_images_in_batches(image_paths, batch_size=32):
    features = []
    start_time = time.time()  # Start time
    for batch_start in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[batch_start:batch_start + batch_size]
        print(f"Processing batch {batch_start // batch_size + 1} of size {len(batch_paths)}")  # Debugging
        batch_images = prepare_images(batch_paths)
        if batch_images is not None and len(batch_images) > 0:
            batch_features = model.predict(batch_images)
            print(f"Batch features shape: {batch_features.shape}")
            features.append(batch_features)
    end_time = time.time()  # End time
    print(f"Total processing time: {end_time - start_time} seconds")
    return np.vstack(features) if len(features) > 0 else None


**5. Train a Model to Map Features to Entity Values - Light GBM**


In [ ]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import boto3
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from io import BytesIO
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# AWS S3 client setup
s3 = boto3.client('s3',
                  aws_access_key_id='Your-access',
                  aws_secret_access_key='Your-secret access')

bucket_name = 'bucket name'
prefix = 'train_images/'  # Folder where images are stored

# Load CSV from S3
train_df = pd.read_csv('s3://bucket name/train.csv')

# Load image from S3
def load_image_from_s3(image_path, target_size=(224, 224)):
    try:
        img_obj = s3.get_object(Bucket=bucket_name, Key=image_path)
        img = load_img(BytesIO(img_obj['Body'].read()), target_size=target_size)
        img_array = img_to_array(img)
        img_array = img_array / 255.0  # normalize
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Prepare image paths
def prepare_images(image_paths):
    images = []
    for path in image_paths:
        img = load_image_from_s3(path)
        if img is not None:
            images.append(img)
    return np.array(images)

# ResNet50 Feature Extractor
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
model = Model(inputs=base_model.input, outputs=x)

# Prepare image paths
image_paths = [prefix + str(group_id) + '_' + str(idx) + '.jpg' for idx, group_id in enumerate(train_df['group_id'])]

# Prepare images and extract features
train_images = prepare_images(image_paths)

# Ensure features are extracted properly
if train_images is not None and len(train_images) > 0:
    # Extract features
    image_features = model.predict(train_images)

    # Flatten features
    image_features = np.reshape(image_features, (image_features.shape[0], -1))

    # Map the features to entity_value (target labels)
    train_labels = train_df['entity_value'].values

    # Split data
    X_train, X_val, y_train, y_val = train_test_split(image_features, train_labels, test_size=0.2, random_state=42)

    # Prepare data for LightGBM
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    # Train the model
    params = {
        'objective': 'regression',
        'metric': 'l2',
        'boosting_type': 'gbdt',
    }
    gbm = lgb.train(params, train_data, valid_sets=[val_data], num_boost_round=100)

    # Predict on validation set
    y_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)

    # Save results to CSV
    output_df = pd.DataFrame({'index': range(len(y_pred)), 'prediction': y_pred})
    output_df.to_csv('validation_predictions.csv', index=False)

else:
    print("No features extracted. Please check the processing steps.")


In [ ]:
print(train_images.shape)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)


In [ ]:
# Train LightGBM Model (or any regression model)
import lightgbm as lgb

# Prepare data for LightGBM
train_data = lgb.Dataset(image_features, label=train_labels)

# Train the model
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 31
}
model_lgb = lgb.train(params, train_data, num_boost_round=100)

# Load test.csv and prepare test image paths
test_df = pd.read_csv(f's3://{bucket_name}/test.csv')
test_image_paths = [prefix + str(group_id) + '_' + str(idx) + '.jpg' for idx, group_id in enumerate(test_df['group_id'])]

# Preprocess test images
test_images = prepare_images(test_image_paths)

# Extract features from the test images
test_image_features = model.predict(test_images)

# Flatten and convert test features to array
test_image_features = np.reshape(test_image_features, (test_image_features.shape[0], -1))

# Make predictions on the test set
predictions = model_lgb.predict(test_image_features)

# Save predictions to CSV
output_df = pd.DataFrame({'index': test_df['index'], 'prediction': predictions})
output_df.to_csv('predictions.csv', index=False)

# Optionally upload the predictions CSV to S3
s3.upload_file('predictions.csv', bucket_name, 'predictions.csv')


**6. Evaluation - F1 Score**

In [ ]:
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_pred, average='macro')  # or 'weighted'
print(f"F1 Score: {f1}")


In [ ]:
# Predict on validation set
y_pred = model.predict(X_val)

# Get the predicted class labels
y_pred_labels = np.argmax(y_pred, axis=1)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred_labels)
print(f"Validation Accuracy: {accuracy}")

# Print classification report
report = classification_report(y_val, y_pred_labels)
print(report)


**7. Save Prediction into csv**

In [ ]:
import pandas as pd

# Assuming you have predictions for your test data
test_predictions = model.predict(X_test)  # Replace X_test with actual test feature data
test_predictions_labels = np.argmax(test_predictions, axis=1)

# Create a DataFrame with predictions
submission_df = pd.DataFrame({
    'image_id': test_image_ids,  # Replace with actual image IDs
    'prediction': test_predictions_labels
})

# Save to CSV
submission_df.to_csv('predictions.csv', index=False)
